In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd

In [ ]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
training_data = []
with open('/content/drive/MyDrive/clean_data_paris_marseille.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']


In [ ]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i, goal_coord,data):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    winds = data[time:time+dtime,:,:,:]
    winds = np.transpose(winds,(0,4,1,2,3))
    winds = np.arctan2(winds[:,0:1,:,:,:],winds[:,1:2,:,:,:])/(3.1415/2)
    pos = np.zeros((dtime,1,winds.shape[2],winds.shape[3],winds.shape[4]))
    for i in range(pos.shape[2]):
        pos[0,0,i,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -(3)
        pos[0,0,i,long,lat] = 3
    input = np.concatenate((winds,pos),axis=1)
    action = np.arctan2(data[time,alt,long,lat,1],data[time,alt,long,lat,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/45)%8)


    return (input,action)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.data = data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.data)
        inp = np.pad(inp,((0,0),(0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [ ]:
coords_destination = [5.3698, 43.2965]
trainset = CustomDataset(training_data,1,coords_destination,data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)


torch.Size([32, 1, 2, 17, 148, 73])


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out

class ProjectedResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ProjectedResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 2, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.projector = nn.AvgPool2d(kernel_size, stride = 2 , padding = padding)
        self.Conv1x1 = nn.Conv2d(in_channels, out_channels, 1, 1, 0)
        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x

        identity = self.projector(x)
        identity =  self.Conv1x1(identity)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += identity
        x = self.relu(x)
        x = self.dropout(x)
        return x

In [ ]:
class Net(nn.Module):
    def __init__(self , print_shape=False):
        self.print_shape = print_shape
        dropoutr = 0.2
        super(Net, self).__init__()
        self.layers1 = nn.ModuleList( [
        nn.Conv3d(2, 64, (17,1,1)),
        nn.BatchNorm3d(64),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Conv3d(64, 64, (1,3,3), padding=(0,1,1)),
        nn.BatchNorm3d(64),
        nn.ReLU(),
        nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))])
        self.layersRES = nn.ModuleList( [ResBlock(64,64, (3,3), 1, 1, dropoutr),
        #ResBlock(64,64, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(64,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(128,256, (3,3), 1, 1, dropoutr),
        #(256,256, (3,3), 1, 1, dropoutr),
        ResBlock(256,256, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(256,512, (3,3), 1, 0, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        nn.Conv2d(512, 512, kernel_size = (2,2), stride = (2,2)),
        nn.BatchNorm2d(512),
        nn.Dropout(dropoutr)])

        self.layers = nn.ModuleList([
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(512, 8)])

        for l in self.layers1:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()


        for l in self.layers:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv2d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:,:]
        for l in self.layers1:
            x = l(x)
            if  (isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or  isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = x[:,:,0,:,:]
        for l in self.layersRES :
            x = l(x)
            if  (isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or  isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

In [ ]:
net = Net().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-45-a8329f902951>:63: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [ ]:
net = net.to(dev)

In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
r = net.to(dev)
r = net(a).to(dev)
b = b.to(dev)
z = torch.argmax(torch.softmax(r,1), axis=1).to(dev)
print(torch.softmax(r,1))
print(z)
print(b - z)
# print(r.data)
# print(b)


tensor([[0.3027, 0.1393, 0.0577, 0.1098, 0.1112, 0.0174, 0.0500, 0.2120],
        [0.3631, 0.0262, 0.1975, 0.0703, 0.0170, 0.2923, 0.0014, 0.0322],
        [0.1272, 0.0726, 0.0729, 0.1741, 0.2923, 0.0899, 0.1321, 0.0388],
        [0.0342, 0.0459, 0.1184, 0.3317, 0.2633, 0.1008, 0.0711, 0.0346],
        [0.0483, 0.0304, 0.6387, 0.0645, 0.0406, 0.0560, 0.0698, 0.0516],
        [0.2268, 0.0188, 0.2454, 0.0992, 0.0401, 0.2288, 0.0421, 0.0987],
        [0.2115, 0.2047, 0.1015, 0.0406, 0.0981, 0.1292, 0.1459, 0.0683],
        [0.0865, 0.1569, 0.0661, 0.0601, 0.3374, 0.0833, 0.1828, 0.0269],
        [0.1017, 0.0335, 0.2086, 0.1127, 0.3812, 0.1023, 0.0074, 0.0526],
        [0.0924, 0.1159, 0.1283, 0.1303, 0.1286, 0.2795, 0.0408, 0.0841],
        [0.2839, 0.1696, 0.1251, 0.0800, 0.2678, 0.0492, 0.0116, 0.0127],
        [0.1910, 0.1667, 0.0402, 0.0571, 0.1592, 0.2745, 0.0807, 0.0304],
        [0.0843, 0.1208, 0.0405, 0.1102, 0.1163, 0.1756, 0.3321, 0.0201],
        [0.1289, 0.0719, 0.0618, 0.174

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
    for i, data in enumerate(trainloader, 0):
      try:
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 10000 == 9999:    # print every 2000 mini-batches
               torch.save(net, "/content/drive/MyDrive/NET2.pt")
      except Exception as e:
        print(e)



print('Finished Training')

[1,   200] loss: 2.245
[1,   400] loss: 2.147
[1,   600] loss: 2.065
[1,   800] loss: 1.984
[1,  1000] loss: 1.955
[1,  1200] loss: 1.895
[1,  1400] loss: 1.836
[1,  1600] loss: 1.826
[1,  1800] loss: 1.789
[1,  2000] loss: 1.774
[1,  2200] loss: 1.718
[1,  2400] loss: 1.725
[1,  2600] loss: 1.707
[1,  2800] loss: 1.661
[1,  3000] loss: 1.664
[1,  3200] loss: 1.615
[1,  3400] loss: 1.620
[1,  3600] loss: 1.581
[1,  3800] loss: 1.576
[1,  4000] loss: 1.560
[1,  4200] loss: 1.520
[1,  4400] loss: 1.537
[1,  4600] loss: 1.512
[1,  4800] loss: 1.486
[1,  5000] loss: 1.492
[1,  5200] loss: 1.466
[1,  5400] loss: 1.475
[1,  5600] loss: 1.434
[1,  5800] loss: 1.429
[1,  6000] loss: 1.443
[1,  6200] loss: 1.433
[1,  6400] loss: 1.411
[1,  6600] loss: 1.400
[1,  6800] loss: 1.385
[1,  7000] loss: 1.369
[1,  7200] loss: 1.365
[1,  7400] loss: 1.361
[1,  7600] loss: 1.355
[1,  7800] loss: 1.318
[1,  8000] loss: 1.325
[1,  8200] loss: 1.323
[1,  8400] loss: 1.303
[1,  8600] loss: 1.270
[1,  8800] 

KeyboardInterrupt: 

In [ ]:
net = torch.load("compressed2.pt")
net = net.to(dev)

In [ ]:
torch.save(net.state_dict, "compressed2.pt")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(100):
        data = next(iter(trainloader))
        images, labels = data
        images = images.float().to(dev)
        labels = labels.to(dev)
        outputs = net(images)
        predicted = torch.argmax(torch.softmax(outputs.data, 1),axis =1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network : %d %%' % (
    100 * correct / total))

Accuracy of the network : 95 %
